# **Forward and Backward Passes**

In this notebook we'll be looking at setting up the forward and backward passes for the fully connected model architecture.

## **Colab Setup**
The setup structure for this will depend on the environment. I'm assuming a Google Colab environment in this case, which will require the following setup from the Github repo, and assumes that the repo has already been cloned into Google Drive:

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/git_folder/DL_From_Foundations
! git pull

Mounted at /content/gdrive
/content/gdrive/My Drive/git_folder/DL_From_Foundations
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/BHouwens/DL_From_Foundations
   e9806a4..ad10e4f  main       -> origin/main
Updating e9806a4..ad10e4f
Fast-forward
 02_fully-connected.ipynb | 299 ++++++++++++++++++++++++++++++++++++-----------
 1 file changed, 234 insertions(+), 65 deletions(-)


In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline



---



## **Normalize Data**

In [3]:
#export
from exp.nb_01 import *
from keras.datasets import mnist

def get_data():
    # Load the data into the train and validation sets
    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()

    # Map the sets to tensors
    x_train,y_train,x_valid,y_valid = map(tensor, (x_train,y_train,x_valid,y_valid))

    # Flatten the 28 * 28 to match the course
    x_train = torch.flatten(x_train, 1)
    x_valid = torch.flatten(x_valid, 1)

    return x_train, x_valid, y_train, y_valid

def normalize(x, m, s): return (x-m)/s

In [4]:
x_train,y_train,x_valid,y_valid = get_data()

train_mean,train_std = x_train.float().mean(),x_train.float().std()
train_mean,train_std

11501568/11490434 [==============================] - 0s 0us/step


(tensor(33.3184), tensor(78.5675))

In [5]:
x_train = normalize(x_train, train_mean, train_std)

# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(1.8892e-08), tensor(1.))

In [8]:
# Let's check out the shapes
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(60000, 784, tensor(0, dtype=torch.uint8))

In [12]:
x_valid.shape

torch.Size([60000])

Now we're talking!



---



## **Foundations (v1.0)**

In [7]:
# num hidden nodes
nh = 50

In [9]:
# 2 layers, so we need 2 weights and 2 biases
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [10]:
def linear_layer(x, w, b):
  return x@w + b

In [ ]:
t = linear_layer(x_valid, w1, b1)